In [ ]:
cd ..

In [ ]:
import echofilter.raw.loader

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
ROOT_DATA_DIR = "/media/scott/scratch/Datasets/dsforce/surveyExports"

# example with 1 passive period, 1 turbulence cut out
sample = "mobile/Survey16/Survey16_GR3_N1W_E"
# example with lots of short passive periods, 1 proper passive period, 1 turbulence cut out
# sample = 'mobile/Survey17/Survey17_GR1_N0W_E'

fname_raw = os.path.join(ROOT_DATA_DIR, sample + "_Sv_raw.csv")
fname_masked = os.path.join(ROOT_DATA_DIR, sample + "_Sv.csv")

ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(fname_raw)
ts_mskd, depths_mskd, signals_mskd = echofilter.raw.loader.transect_loader(fname_masked)

t_bot, d_bot = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_bottom.evl")
)
t_top, d_top = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_turbulence.evl")
)

In [ ]:
np.min(signals_raw), np.max(signals_raw)

In [ ]:
ts_raw.shape, depths_raw.shape, signals_raw.shape

In [ ]:
turbulence_color = "c"
bottom_color = "#00ee00"

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, signals_raw.T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.show()

## Finding boxes to merge

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, ~np.isnan(signals_mskd).T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
indices = np.tile(np.arange(signals_mskd.shape[1]), (signals_mskd.shape[0], 1)).astype(
    "float"
)
indices[np.isnan(signals_mskd)] = np.nan

In [ ]:
np.nanmin(indices, axis=1)

In [ ]:
np.nanmax(indices, axis=1)

In [ ]:
cut_off_indices = np.round(np.nanmedian(indices, axis=1))

In [ ]:
cut_off_indices

In [ ]:
depths_mskd

In [ ]:
top_depths = np.tile(
    np.concatenate([(depths_mskd[:-1] + depths_mskd[1:]) / 2, depths_mskd[-1:]]),
    (signals_mskd.shape[0], 1),
)
top_depths[~np.isnan(signals_mskd)] = np.nan

In [ ]:
top_depths

In [ ]:
np.expand_dims(np.arange(signals_mskd.shape[1]), 0)

In [ ]:
np.expand_dims(cut_off_indices, -1)

In [ ]:
plt.imshow(top_depths.T)

In [ ]:
li = np.expand_dims(np.arange(signals_mskd.shape[1]), 0) > np.expand_dims(
    cut_off_indices, -1
)
plt.imshow(li.T)

In [ ]:
top_depths[li] = np.nan
plt.imshow(top_depths.T)

In [ ]:
d_top_new = np.nanmax(top_depths, axis=1)

In [ ]:
bot_depths = np.tile(
    np.concatenate([depths_mskd[:1], (depths_mskd[:-1] + depths_mskd[1:]) / 2]),
    (signals_mskd.shape[0], 1),
)
bot_depths[~np.isnan(signals_mskd)] = np.nan
li = np.expand_dims(np.arange(signals_mskd.shape[1]), 0) < np.expand_dims(
    cut_off_indices, -1
)
bot_depths[li] = np.nan
d_bot_new = np.nanmin(bot_depths, axis=1)

In [ ]:
plt.imshow(bot_depths.T)

In [ ]:
plt.plot(d_top_new)
plt.plot(d_bot_new)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(ts_mskd, d_top_new, turbulence_color)
plt.plot(ts_mskd, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
d_bot

In [ ]:
d_bot_new

In [ ]:
ts_mskd.shape

In [ ]:
t_bot.shape

In [ ]:
plt.plot(t_bot, d_bot)
plt.plot(ts_mskd, d_bot_new)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.plot(t_top, d_top)
plt.plot(ts_mskd, d_top_new)
plt.gca().invert_yaxis()
plt.show()

## Finding rows which are fully removed from masked output

In [ ]:
np.all(np.isnan(signals_mskd), axis=1)

In [ ]:
allnan = np.all(np.isnan(signals_mskd), axis=1)
removed_starts = np.nonzero(np.diff(allnan.astype(np.float)) > 0)[0] + 1
removed_ends = np.nonzero(np.diff(allnan.astype(np.float)) < 0)[0] + 1

if removed_ends[0] < removed_starts[0]:
    removed_starts = np.concatenate(([0], removed_starts))

if removed_starts[-1] > removed_ends[-1]:
    removed_ends = np.concatenate((removed_ends, [len(allnan)]))

In [ ]:
print(removed_starts)
print(removed_ends)

In [ ]:
for index_start, index_end in zip(removed_starts, removed_ends):
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(
        ts_raw[index_start:index_end],
        depths_raw,
        signals_raw[index_start:index_end, :].T,
    )
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
for index_start, index_end in zip(
    np.concatenate(([0], removed_ends)),
    np.concatenate((removed_starts, [signals_raw.shape[0]])),
):
    if index_start == index_end:
        continue
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(
        ts_raw[index_start:index_end],
        depths_raw,
        signals_raw[index_start:index_end, :].T,
    )
    plt.gca().invert_yaxis()
    plt.show()